# CCLIB + Molden scratch
03/02/21

Working on Stimpy, from previous tests, see http://localhost:8888/lab/tree/Scratch/epsman-bemo-AntonJr_March2020/N2O_electronic_structure_proc_tests_250820.ipynb

Also http://localhost:8889/lab/tree/Scratch/epsman-bemo-AntonJr_March2020/N2O_electronic_structure_proc_tests_250820_revisit-010221.ipynb

Two approaches:

- Build/modify existing CCLIB class to change output format at file write.
- Read & modify existing file to fit ePS formatting requirements.

Both could be useful, both now tested in principle, but formatting not finalised.

In [2]:
!conda env list

# conda environments:
#
                         C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64
                         C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\dataVis3D
                         C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\dataVis3D-yt
                         C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\ePSdev
                         C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\ePSpkgTest2
                         C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\fibre-sim
                         C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\ipykernel_py2
                         C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\pkgTest
                         C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\pypi-test
                        

## Set and read file

In [10]:
from pathlib import Path

# fileName = Path(r"/home/femtolab/ePS/N2O/electronic_structure/N2O_aug-cc-pVDZ_geomOpt.log")  # N2O Gamess file  OK
fileName = Path(r"D:\VMs\Share\ePSshare\N2O\electronic_structure\N2O_aug-cc-pVDZ_geomOpt.log")  # N2O Gamess file  OK


In [11]:
import cclib

# parser = cclib.io.ccopen(filename)
# data = parser.parse()
# print("There are %i atoms and %i MOs" % (data.natom, data.nmo))

# Alt form - newer, so should be preferred.
data = cclib.io.ccread(fileName.as_posix())
print("There are %i atoms and %i MOs" % (data.natom, data.nmo))

There are 3 atoms and 69 MOs


## Write Molden format file

In [4]:
fileName.with_suffix('.test030221.molden')

WindowsPath('D:/VMs/Share/ePSshare/N2O/electronic_structure/N2O_aug-cc-pVDZ_geomOpt.test030221.molden')

In [6]:
# Convert to Molden format
f = 'molden'  # Set output format
outFile = fileName.with_suffix('.test030221.' + f)
cclib.io.ccwrite(data, terse=True, outputtype=f, outputdest=outFile.as_posix())  # From data

As compared to file converted via Molden directly (v5.9):

- File stops at end of [MO] section. (Doesn't include SCF results.)
- Symmetry listed for each orbital - these seem to be missing from vanilla Molden output.
- (x,y,z) coords tidied up a little, but basically same format - will need fixing for ePS input.

For more details:

- https://github.com/cclib/cclib/blob/master/cclib/io/moldenwriter.py
- https://github.com/cclib/cclib/blob/master/cclib/io/ccio.py




In [7]:
# Call without filename - returns representation as string
f = 'molden'  # Set output format
mOutput = cclib.io.ccwrite(data, terse=True, outputtype=f)  # From data

## Test subfunctions

In [14]:
from cclib.io import moldenwriter  # Molden class + functions
# from cclib.io import MOLDEN  # Molden class
# MOLDEN

In [18]:
# Try some subfunctions... may be able to wrap/decorate/modify these as required?
molData = moldenwriter.MOLDEN(data)  # Create class object from cclib object
molData._coords_from_ccdata(-1)

['O      1  8     0.000000     0.000000    -1.087767',
 'N      2  7     0.000000     0.000000     0.087116',
 'N      3  7     0.000000     0.000000     1.179311']

In [20]:
molData._gto_from_ccdata()

['  1 0',
 's     8 1.00',
 '1.172000000e+04 7.096459470e-04',
 '1.759000000e+03 5.467272292e-03',
 '4.008000000e+02 2.782311861e-02',
 '1.137000000e+02 1.047477397e-01',
 '3.703000000e+01 2.829208464e-01',
 '1.327000000e+01 4.484952387e-01',
 '5.025000000e+00 2.708168852e-01',
 '1.013000000e+00 1.545029161e-02',
 's     8 1.00',
 '1.172000000e+04 -3.144434120e-04',
 '1.759000000e+03 -2.482137681e-03',
 '4.008000000e+02 -1.231635538e-02',
 '1.137000000e+02 -5.053891734e-02',
 '3.703000000e+01 -1.393849033e-01',
 '1.327000000e+01 -3.250774948e-01',
 '5.025000000e+00 -2.298483076e-01',
 '1.013000000e+00 1.095379347e+00',
 's     1 1.00',
 '3.023000000e-01 1.000000000e+00',
 'p     3 1.00',
 '1.770000000e+01 6.267916628e-02',
 '3.854000000e+00 3.335365659e-01',
 '1.046000000e+00 7.412396416e-01',
 'p     1 1.00',
 '2.753000000e-01 1.000000000e+00',
 'd     1 1.00',
 '1.185000000e+00 1.000000000e+00',
 's     1 1.00',
 '7.896000000e-02 1.000000000e+00',
 'p     1 1.00',
 '6.856000000e-02 1

In [41]:
# Try redefining existing methods

# Slight modification of cclib source
def _coords_from_ccdata(self):
    lines = super()  # Just use super here? Can't remember how to test this when monkey patching however - doesn't resolve properly?
#     super()
    
    linesReformat = []
    for line in lines:
        linesReformat.append('{:>3} {:>5} {:>5} {:>20} {:>20} {:>20}'.format(*line.split()))
    
    return linesReformat
    
    
# Code from cclib, with minor mods, see https://github.com/cclib/cclib/blob/bbc231295e64c7f25d5d235492c103688a4e068b/cclib/io/moldenwriter.py#L68
def _gto_from_ccdata(self):
    """Create [GTO] section using gbasis.
    atom_sequence_number1 0
    shell_label number_of_primitives 1.00
    exponent_primitive_1 contraction_coeff_1 (contraction_coeff_1)
    ...
    empty line
    atom_sequence__number2 0
    """

    gbasis = self.ccdata.gbasis
    label_template = ' {:s} {:5d} 1.00'     # Changed spacing here - single space for basis label, three spaces for components.
    basis_template = '   {:15.9e} {:15.9e}'
    lines = []

    for no, basis in enumerate(gbasis):
        lines.append('{:3d} 0'.format(no + 1))
        for prims in basis:
            lines.append(label_template.format(prims[0].lower(),
                                               len(prims[1])))
            for prim in prims[1]:
                lines.append(basis_template.format(prim[0], prim[1]))
                
                lines[-1] = lines[-1].replace('e','D')  # Added for D format output - may not be required
                
                
        lines.append('')
    lines.append('')
    return lines

# Monkey patch for new routines
# See, e.g., https://stackoverflow.com/a/2982
import types

molData._coords_from_ccdata = types.MethodType(_coords_from_ccdata, molData)
molData._gto_from_ccdata = types.MethodType(_gto_from_ccdata, molData)

In [40]:
molData._coords_from_ccdata()

RuntimeError: super(): __class__ cell not found

In [35]:
molData._gto_from_ccdata()

['  1 0',
 ' s     8 1.00',
 '   1.172000000D+04 7.096459470D-04',
 '   1.759000000D+03 5.467272292D-03',
 '   4.008000000D+02 2.782311861D-02',
 '   1.137000000D+02 1.047477397D-01',
 '   3.703000000D+01 2.829208464D-01',
 '   1.327000000D+01 4.484952387D-01',
 '   5.025000000D+00 2.708168852D-01',
 '   1.013000000D+00 1.545029161D-02',
 ' s     8 1.00',
 '   1.172000000D+04 -3.144434120D-04',
 '   1.759000000D+03 -2.482137681D-03',
 '   4.008000000D+02 -1.231635538D-02',
 '   1.137000000D+02 -5.053891734D-02',
 '   3.703000000D+01 -1.393849033D-01',
 '   1.327000000D+01 -3.250774948D-01',
 '   5.025000000D+00 -2.298483076D-01',
 '   1.013000000D+00 1.095379347D+00',
 ' s     1 1.00',
 '   3.023000000D-01 1.000000000D+00',
 ' p     3 1.00',
 '   1.770000000D+01 6.267916628D-02',
 '   3.854000000D+00 3.335365659D-01',
 '   1.046000000D+00 7.412396416D-01',
 ' p     1 1.00',
 '   2.753000000D-01 1.000000000D+00',
 ' d     1 1.00',
 '   1.185000000D+00 1.000000000D+00',
 ' s     1 1.00',

#### Try sitting on class with inheritance

In [49]:
# Try redefining existing methods - class version with inheritance

class moldenReformatted(moldenwriter.MOLDEN):
    # Slight modification of cclib source
    def _coords_from_ccdata(self, index):
        lines = super()._coords_from_ccdata(index)  # Just use super here? Can't remember how to test this when monkey patching however - doesn't resolve properly?
    #     super()

        linesReformat = []
        for line in lines:
            linesReformat.append('{:>3} {:>5} {:>5} {:>20} {:>20} {:>20}'.format(*line.split()))

        return linesReformat


    # Code from cclib, with minor mods, see https://github.com/cclib/cclib/blob/bbc231295e64c7f25d5d235492c103688a4e068b/cclib/io/moldenwriter.py#L68
    def _gto_from_ccdata(self):
        """Create [GTO] section using gbasis.
        atom_sequence_number1 0
        shell_label number_of_primitives 1.00
        exponent_primitive_1 contraction_coeff_1 (contraction_coeff_1)
        ...
        empty line
        atom_sequence__number2 0
        """

        gbasis = self.ccdata.gbasis
        label_template = ' {:s} {:5d} 1.00'     # Changed spacing here - single space for basis label, three spaces for components.
        basis_template = '   {:15.9e} {:15.9e}'
        lines = []

        for no, basis in enumerate(gbasis):
            lines.append('{:3d} 0'.format(no + 1))
            for prims in basis:
                lines.append(label_template.format(prims[0].lower(),
                                                   len(prims[1])))
                for prim in prims[1]:
                    lines.append(basis_template.format(prim[0], prim[1]))

                    lines[-1] = lines[-1].replace('e','D')  # Added for D format output - may not be required


            lines.append('')
        lines.append('')
        return lines

    
    
# # Monkey patch for new routines
# # See, e.g., https://stackoverflow.com/a/2982
# import types

# molData._coords_from_ccdata = types.MethodType(_coords_from_ccdata, molData)
# molData._gto_from_ccdata = types.MethodType(_gto_from_ccdata, molData)

In [51]:
molDataRF = moldenReformatted(data)  # Create class object from cclib object
molDataRF._coords_from_ccdata(-1)

['  O     1     8             0.000000             0.000000            -1.087767',
 '  N     2     7             0.000000             0.000000             0.087116',
 '  N     3     7             0.000000             0.000000             1.179311']

In [53]:
# molDataRF.generate_repr()
dir(molDataRF)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_calculate_total_dipole_moment',
 '_check_required_attributes',
 '_coords_from_ccdata',
 '_fix_indices',
 '_gto_from_ccdata',
 '_make_bond_connectivity_from_openbabel',
 '_make_openbabel_from_ccdata',
 '_mo_from_ccdata',
 '_rearrange_mocoeffs',
 '_scfconv_from_ccdata',
 '_title',
 'ccdata',
 'generate_repr',
 'ghost',
 'indices',
 'jobfilename',
 'pt',
 'required_attrs',
 'terse']

In [58]:
molDataRF.ccdata

TypeError: 'ccData_optdone_bool' object is not callable

## With epsman class

In [61]:
# Load dev scripts
import sys
# modPath = r'/home/femtolab/python/'
modPath = r'D:\code\github'
sys.path.append(modPath)
# import epsman as em
from epsman.elecStructure.ESclass import EShandler

In [62]:
# fileBase = r'D:\VMs\Share\ePSshare\N2O\electronic_structure'
fileBase = Path(modPath, 'elecStructure','fileTest')
fileName = r'N2O_aug-cc-pVDZ_geomOpt.log'
esData = EShandler(fileName, fileBase)

Could not import `openbabel`, fallback mechanism might not work.
Read file D:\code\github\elecStructure\fileTest\N2O_aug-cc-pVDZ_geomOpt.log


AttributeError: 'NoneType' object has no attribute 'natom'

In [ ]:
Path()